In [41]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# ---------------------------
# Dataset Class
# ---------------------------
class CommunicationDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        label = self.labels[idx]
        return torch.FloatTensor(sequence), torch.FloatTensor([label])


In [42]:

# ---------------------------
# Model
# ---------------------------
class TelemetryLossPredictor(nn.Module):
    def __init__(self, input_dim=5, hidden_dim=64, num_layers=2, output_dim=2, dropout=0.2):
        super(TelemetryLossPredictor, self).__init__()

        # CNN
        self.conv1d = nn.Sequential(
            nn.Conv1d(input_dim, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.AdaptiveAvgPool1d(1)
        )

        # LSTM
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True
        )

        # Attention
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim*2, 64),
            nn.Tanh(),
            nn.Linear(64, 1),
            nn.Dropout(dropout)
        )

        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(64 + hidden_dim*2, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        # CNN path
        cnn_input = x.transpose(1,2)  # (batch, input_dim, seq_len)
        cnn_features = self.conv1d(cnn_input).squeeze(-1)

        # LSTM path
        lstm_out, _ = self.lstm(x)
        attention_weights = torch.softmax(self.attention(lstm_out).squeeze(-1), dim=1)
        lstm_features = torch.sum(lstm_out * attention_weights.unsqueeze(-1), dim=1)

        # Combine
        combined_features = torch.cat([cnn_features, lstm_features], dim=1)
        output = self.classifier(combined_features)

        return output, attention_weights


In [43]:
# ---------------------------
# Feature Engineering
# ---------------------------
class CommunicationFeatureEngineer:
    def __init__(self, window_size=60, prediction_horizon=30):
        self.window_size = window_size
        self.prediction_horizon = prediction_horizon
        self.scaler = StandardScaler()

    def compute_derived_features(self, df):
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        features = [
            df['value'].rolling(5).mean(),
            df['value'].rolling(10).std(),
            df['value'].diff().rolling(5).mean()
        ]
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        features.extend([
            df['timestamp'].dt.second,
            (df['timestamp'].diff().dt.total_seconds().fillna(0) > 2).cumsum()
        ])
        derived_df = pd.concat(features, axis=1).fillna(method='bfill').fillna(method='ffill')
        return derived_df

    def create_sequences(self, data, labels):
        sequences = []
        sequence_labels = []
        for i in range(len(data) - self.window_size - self.prediction_horizon):
            sequences.append(data[i:(i+self.window_size)])
            sequence_labels.append(labels[i+self.window_size+self.prediction_horizon])
        return np.array(sequences), np.array(sequence_labels)


In [44]:
# ---------------------------
# Trainer
# ---------------------------
class TelemetryLossTrainer:
    def __init__(self, model, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.model = model.to(device)
        self.device = device
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, patience=5)

    def train_epoch(self, dataloader):
        self.model.train()
        total_loss = 0
        for data, targets in dataloader:
            data, targets = data.to(self.device), targets.to(self.device)
            self.optimizer.zero_grad()
            outputs, _ = self.model(data)
            loss = self.criterion(outputs, targets.squeeze().long())
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(dataloader)

    def evaluate(self, dataloader):
        self.model.eval()
        total_loss = 0
        all_preds, all_targets = [], []
        with torch.no_grad():
            for data, targets in dataloader:
                data, targets = data.to(self.device), targets.to(self.device)
                outputs, _ = self.model(data)
                loss = self.criterion(outputs, targets.squeeze().long())
                total_loss += loss.item()
                probs = torch.softmax(outputs, dim=1)
                all_preds.extend(probs.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())
        return total_loss / len(dataloader), np.array(all_preds), np.array(all_targets)


In [ ]:
# ---------------------------
# Main Pipeline
# ---------------------------
def main():
    # Load data
    df = pd.read_csv('/content/segments.csv')

    # Encode labels
    df['label_encoded'] = df['label'].map({'anomaly':1, 'normal':0}).fillna(0).astype(int)
    labels = df['label_encoded'].values

    # Feature engineering
    feature_engineer = CommunicationFeatureEngineer(window_size=60, prediction_horizon=30)
    derived_features = feature_engineer.compute_derived_features(df)
    feature_data = derived_features.values
    feature_columns = derived_features.columns.tolist()

    # Create sequences
    sequences, sequence_labels = feature_engineer.create_sequences(feature_data, labels)

    # Split train/validation
    split_idx = int(0.7 * len(sequences))
    train_sequences, val_sequences = sequences[:split_idx], sequences[split_idx:]
    train_labels, val_labels = sequence_labels[:split_idx], sequence_labels[split_idx:]

    # Dataloaders
    train_loader = DataLoader(CommunicationDataset(train_sequences, train_labels), batch_size=32, shuffle=True)
    val_loader = DataLoader(CommunicationDataset(val_sequences, val_labels), batch_size=32, shuffle=False)

    # Model
    model = TelemetryLossPredictor(input_dim=len(feature_columns), output_dim=2)
    trainer = TelemetryLossTrainer(model)

    # Training loop
    best_val_loss = float('inf')
    for epoch in range(20):  # Adjust epochs
        train_loss = trainer.train_epoch(train_loader)
        val_loss, val_preds, val_targets = trainer.evaluate(val_loader)
        val_auc = roc_auc_score(val_targets, val_preds[:,1])  # class 1 AUC
        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val AUC={val_auc:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'model_state_dict': model.state_dict(),
                'feature_columns': feature_columns,
                'val_loss': val_loss,
                'val_auc': val_auc
            }, 'best_communication_model.pth')
        trainer.scheduler.step(val_loss)

if __name__ == "__main__":
    main()